## 文档的创建和实例

In [2]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField

In [3]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), authsource='engine_learn3', uuidrepresentation=3)

### 静态文档和动态文档

In [4]:
# 继承自Document的文档
class Book1(Document):
    bookid = StringField()
    name = StringField()
    count = IntField()
    meta = {"db_alias": TEST_DB1}

In [4]:
# 在静态文档上添加文档中没有的字段,不会被保存在数据库集合的文档中；文档模型中存在的字段可以被保存
book1 = Book1(bookid="wx000008", name="西游记", count=39)
book1.type = "文学著作"
book1.save()

<Book1: Book1 object>

In [5]:
# 动态文档, 可以在原有的数据模式上动态添加字段
from mongoengine import DynamicDocument
class BookDynamic(DynamicDocument):
    bookid = StringField()
    meta = {"db_alias": TEST_DB1}
bookd = BookDynamic(bookid="wx09872")
bookd.tag = ["小说", "经典著作"]
bookd.save()

<BookDynamic: BookDynamic object>

### 关于修改文档类:
前期建立的文档类(静态),并已经建立并使用，之后修改新增字段时，前期使用生成的文档字段中针对后期新增的部分为空值(非null值，而是字段不存在)。  
在原有基础上操作修改后的文档类, 此时并不会报错。

In [10]:
# 修改原来的 Book1字段
class Book1(Document):
    bookid = StringField(unique=True)
    name = StringField()
    count = IntField()
    author = StringField()
    meta = {"db_alias": TEST_DB1}

In [12]:
Book1(name="红楼梦", count=89, author="曹雪芹").save()

<Book1: Book1 object>

In [7]:
Book1.objects.to_json(ensure_ascii=False)

'[{"_id": {"$oid": "63fec2db3ba566dfccb4e2a5"}, "bookid": "wx00swde", "name": "东游记", "count": 20}, {"_id": {"$oid": "640bf32d3018ab8dde81e564"}, "bookid": "wx000006", "name": "水浒传", "count": 29}, {"_id": {"$oid": "640bf3923018ab8dde81e565"}, "bookid": "wx000008", "name": "西游记", "count": 39}, {"_id": {"$oid": "640bf4513018ab8dde81e566"}, "bookid": "wx0000064f", "name": "红楼梦", "count": 9, "author": "曹雪芹"}, {"_id": {"$oid": "640bf6bd90a4848092913a1c"}, "bookid": "wx0000064f", "name": "红楼梦", "count": 9, "author": "曹雪芹"}]'

### 创建文档实例

To create a new document object, create an instance of the relevant document class, providing values for its fields as constructor keyword arguments. You may provide values for any of the fields on the document:

### 保存文档的方法

`Book1(bookid="wx00swde", name="东游记", count=9).save()`

MongoEngine tracks changes to documents to provide efficient saving. To save the document
to the database, call the save() method. If the document does not exist in the database, it
will be created. **If it does already exist, then any changes will be updated atomically**.

已存在保存的正确使用: 需要在保存文档时使用 primary_key, 或 _id 字段值，否则无法判断文档是否已存在。
或者使用objects查询出的文档，修改后再保存即为更新。

In [15]:
# 保存新文档;此时调用Book1类创建了一个新文档，即使内容和已有的文档一致，也会被作为一个新文档保存。
book_dyj = Book1(bookid="wx00swde", name="东游记", count=9).save()

In [17]:
# 查询出已存在的文档，修改属性值后保存，即是更新了原有文档
book_dyj_doc = Book1.objects(name="东游记").first()

In [20]:
book_dyj.count = 20
book_dyj.save()
# 保存后，原文档的数量已经发生变化

<Book1: Book1 object>

### 2.4.1.1. Cascading Saves(级联保存)

`Document.cascade_save(**kwargs)`: Recursively save any references and generic references on the document.

If your document contains `ReferenceField` or `GenericReferenceField` objects, then by default the `save()` method will **not save any changes to those objects**. If you want all references to be saved also, noting each save is a separate query, then passing `cascade` as True to the save method will cascade any saves.

译文:  
如果您的文档包含`ReferenceField` or `GenericReferenceField`对象，则默认情况下该  `save()` 方法**不会保存对这些对象的任何更改**。如果您还希望保存所有引用，请注意每次保存都是一个单独的查询，然后将 `True` 传递给保存方法将级联任何保存。

### 删除文档的方法

`Book1.objects(name="西游记").delete()`

In [6]:
Book1(bookid="wx00000de", name="西游记", count=59).save()

<Book1: Book1 object>

In [9]:
Book1.objects(name="西游记").first()
# 查询结果如下

<Book1: Book1 object>

In [10]:
Book1.objects(name="西游记").delete()
# 删除文档后，返回数量1

1

In [11]:
Book1.objects(name="西游记").first()
# 再次查询后不返回结果

In [12]:
Book1.objects(name="西游记").delete()
# 删除不存在的文档，返回 0

0

In [13]:
res = Book1.objects(name="西游记").first()
print(res)  # 查询不存在的文档，返回 None

None


### 集合 2.3.4. Document collections

Document classes that inherit directly from `Document` will have their own collection in the database. The name of the collection is by **default the name of the class converted to `snake_case`** (e.g if your Document class is named *CompanyUser*, the corresponding collection would be *company_user*). 

If you need to change the name of the collection (e.g. to use MongoEngine with an existing database), then create a class dictionary attribute called `meta` on your document, and set collection to the name of the `collection` that you want your document class to use:

```python
class Page(Document):
    title = StringField(max_length=200, required=True)
    meta = {'collection': 'cmsPage'}
```

**总结**

- 默认的集合名是文档的类名变化而来， 类似的命名规则为: *CompanyUser* -> *company_user*
- 如果想修改集合名称，可以在创建类时 使用 `meta = {'collection': 'cmsPage'}`

### 2.3.6 默认排序

ordering, 在meta中设置`QuerySet`结果以某一字段默认排序  
A *default ordering* can be specified for your `QuerySet` using the `ordering` attribute of `meta`. Ordering will be applied when the `QuerySet` is created, and can be overridden by subsequent calls to `order_by()`.

In [27]:
# 继承自Document的文档
class BookOrdering(Document):
    bookid = StringField()
    name = StringField()
    count = IntField()
    meta = {
        "db_alias": TEST_DB1,
        'ordering': ['+count']  # +： 升序排列，第一个最小； -： 降序排列， 第一个最大
    }
BookOrdering(bookid="wx000006", name="水浒传", count=29).save()
BookOrdering(bookid="wx000007", name="三国演绎", count=9).save()
BookOrdering(bookid="wx000007", name="红楼梦", count=109).save()

<BookOrdering: BookOrdering object>

In [28]:
BookOrdering.objects.to_json()

'[{"_id": {"$oid": "63ff4b583ba566dfccb4e2aa"}, "bookid": "wx000007", "name": "\\u4e09\\u56fd\\u6f14\\u7ece", "count": 9}, {"_id": {"$oid": "63ff4b583ba566dfccb4e2a9"}, "bookid": "wx000006", "name": "\\u6c34\\u6d52\\u4f20", "count": 29}, {"_id": {"$oid": "63ff4b583ba566dfccb4e2ab"}, "bookid": "wx000007", "name": "\\u7ea2\\u697c\\u68a6", "count": 109}]'